In [14]:
import tiktoken

In [15]:
tokenizer = tiktoken.get_encoding("gpt2")

text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunkwownPlace"
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 2954, 86, 593, 27271]


In [16]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunkwownPlace


In [17]:
integers = tokenizer.encode("Akwirw ier")
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


In [18]:
with open("/home/enid/Downloads/wizard-of-oz.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)

In [19]:
context_size = 4

enc_sample = enc_text[50:]
x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]

print(f"x: {x}")
print(f"y:          {y}")

x: [15485, 13, 921, 743]
y:          [13, 921, 743, 4866]


In [20]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

soever ----> .
soever. ---->  You
soever. You ---->  may
soever. You may ---->  copy


In [21]:
from torch.utils.data import Dataset, DataLoader
import torch

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_len, stride):    # max_len = context_size, stride determina quantas palavras para o lado vamos
        self.input_ids = []
        self.target_ids = []
        
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        # Sliding window
        for i in range(0, len(token_ids) - max_len, stride):
            input_chunk = token_ids[i : i + max_len]
            target_chunk = token_ids[i + 1 : i + max_len + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):     # retorna linha idx do tensor de input e linha idx do tensor de output, necessário para colocar no dataloader
        return self.input_ids[idx], self.target_ids[idx]
            

In [22]:
# drop_last = True para retirar ultimo batch caso este seja menor que batch_size para prevenir spikes de loss no treinamento
# Ajuda a criar pares input output a partir do dataset que definimos anteriormente
# Facilita processamento em paralelo
# batch_size é o número de batches que o modelo processa antes de atualizar seus parâmetros
# num_workers é o número de cpus para processar
def create_dataloader_v1(txt, batch_size=4, max_len=256,        # batch_size -> numero de cpus para rodar
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    
    tokenizer = tiktoken.get_encoding("gpt2")
    
    # Inicializando dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_len, stride)
    
    # Inicializando dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    
    return dataloader

In [23]:
with open("/home/enid/Downloads/wizard-of-oz.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [24]:
dataloader = create_dataloader_v1(
    text, batch_size=1, max_len=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[171, 119, 123, 464]]), tensor([[ 119,  123,  464, 4935]])]


In [25]:
# Mudando o batch_size para 8 -> 8 tensores de input e 8 de output para processar
# Colocando o stride para 4, utilizamos o dataset em sua totalidade e prevenimos sobreposição de palavras de cada batch. A sobreposição poderia aumentar o overfitting
# Manter stride igual a max_len (contexto)
dataloader = create_dataloader_v1(text, batch_size=8, max_len=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Inputs:\n", inputs)
print("Targets:\n", targets)

Inputs:
 tensor([[  171,   119,   123,   464],
        [ 4935, 20336, 46566,   286],
        [  383, 40128, 16884,   286],
        [18024,   198,   220,   220],
        [  220,   220,   198,  1212],
        [47179,   318,   329,   262],
        [  779,   286,  2687,  6609],
        [  287,   262,  1578,  1829]])
Targets:
 tensor([[  119,   123,   464,  4935],
        [20336, 46566,   286,   383],
        [40128, 16884,   286, 18024],
        [  198,   220,   220,   220],
        [  220,   198,  1212, 47179],
        [  318,   329,   262,   779],
        [  286,  2687,  6609,   287],
        [  262,  1578,  1829,   290]])


In [26]:
input_ids = torch.tensor([2, 3, 5, 1]) # Token ids, cada um associado a uma palavra
vocab_size = 6
output_dim = 3 # Dimensões do vetor

torch.manual_seed(123) # Gerar sempre os mesmos números
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)    # Inicia pesos com valores aleatórios

print(embedding_layer.weight) # Cada linha é o vector embedding de cada token

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [27]:
# Representação dos pesos do vetor com o token id 3
# Embedding layer serve como uma tabela para olhar pesos 
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [ ]:
print(embedding_layer(input_ids))   # Olha a tabela de pesos dos ids passados